In [71]:

import os
import json
import datetime
import dateutil
import requests
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from dateutil.parser import parse
from datetime import datetime, timedelta

%matplotlib inline

# I/O
## Make a directory


In [4]:
# Does the Data directory exist?
if not os.path.exists("./Data"):
    os.mkdir("./Data")
    print("'Data' directory created!")
else:
    print("'Data' directory already exists!")


'Data' directory already exists!


Let's create a new pandas dataframe and store it in the 'Data' directory

In [5]:
values = np.array([
    [100, 80, 95, 'A'],
    [55, 60, 45, 'F'],
    [70, 75, 90, 'A'],
    [75, 70, 60, 'D'],
    [60, 73, 75, 'C'],
    [72, 63, -1, 'NA']
])
df = pd.DataFrame(values,
                   columns=['Midterm', 'Project', 'Final', 'LetterGrade'],
                   index=['Alex', 'Bob', 'Chris', 'Doug', 'Eva', "Frank"])
df

,Midterm,Project,Final,LetterGrade
Alex,100,80,95,A
Bob,55,60,45,F
Chris,70,75,90,A
Doug,75,70,60,D
Eva,60,73,75,C
Frank,72,63,-1,NA


In [6]:
# Does the file exist?
if not os.path.exists("Data/io_test.csv"):
    df.to_csv("Data/io_test.csv")
    print("'io_test.csv' created!")
else:
    print("'io_test.csv' already exists!")

'io_test.csv' already exists!


Now let's load the dataframe

In [7]:
df2 = pd.read_csv("Data/io_test.csv", sep=",")

There are some arguments that can be passed to `read_csv`:

- **nrows** `int`: number of rows to be loaded.
- **skiprows** `[int]` : index of rows to be skipped.
- **names** : `[str]` : specify the column names for the dataframe.
- **index_col** `int` : index of the column in the dataframe to be treated as the index column.
- **sep** `str` : dataframe delimiter (_alias_ : _delimeter_). This column accepts literal characters or regular expressions.


### Load JSON files

In [8]:
# JSON string 
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [9]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

### Convert a python object into a JSON object 

In [10]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [11]:
# Use json.dump(object, file) to write the content to file

# This is called a context manager
with open("Data/People.json", 'w') as file:
    json.dump(result, file)

In [12]:
# Load from People.json
with open("Data/People.json", "r") as file:
    people = json.load(file)
people

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

### Load JSON content into a pandas Dataframe

In [13]:
# Load the content as a data frame
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


# Data Transformations

In [14]:
# Let's create a dataframe with duplicate data
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [15]:
# Identify duplicated rows
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [16]:
# Drop duplicated rows
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


# Map one column to another and add it to the dataframe

In [17]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [18]:
# Suppose that we want to map the meat type to the kind of animal:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [19]:
# To make matching simpler, change strings to lowercase first
lowercased = data['food'].str.lower()
# lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


# Join and Combine

In [20]:
homework = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare', 'David'],
    'Hw1': [100, 90, 80, 70],
    'Hw2': [60, 70, 80, 90]
})
homework

,Name,Hw1,Hw2
0,Alice,100,60
1,Bob,90,70
2,Clare,80,80
3,David,70,90


In [21]:
exam = pd.DataFrame({
    "Full Name": ['Alice', 'Bob', 'Clare', 'Eli'],
    "Midterm": [70, 80, 90, 100],
    "Final": [85, 65, 75, 55]
})
exam

,Full Name,Midterm,Final
0,Alice,70,85
1,Bob,80,65
2,Clare,90,75
3,Eli,100,55


In [22]:
# Default merge will drop values that cannot find a match
pd.merge(homework, exam,
         left_on="Name",
         right_on="Full Name")

,Name,Hw1,Hw2,Full Name,Midterm,Final
0,Alice,100,60,Alice,70,85
1,Bob,90,70,Bob,80,65
2,Clare,80,80,Clare,90,75


The `left_on` and `right_on` arguments specify which columns should pandas use to merge the dataframes. For instance, in the above cell we have `left_on="Name"` and `right_on="Full Name"`. This means that pandas will use the _"Name"_ column from the `homework` dataframe, and the _"Full Name"_ columns from the `exam` dataframe.

**Note:** `left_on` and `right_on` can take either a `str` or `[str]` 


In [23]:
pd.merge(homework, exam, left_on="Name", right_on="Full Name",
         how='outer')

,Name,Hw1,Hw2,Full Name,Midterm,Final
0,Alice,100.0,60.0,Alice,70.0,85.0
1,Bob,90.0,70.0,Bob,80.0,65.0
2,Clare,80.0,80.0,Clare,90.0,75.0
3,David,70.0,90.0,NaN,NaN,NaN
4,NaN,NaN,NaN,Eli,100.0,55.0


The `how` arguments specifies how pandas should merge the passed dataframes. Here are the types of merges:

- `inner`: Use only the keys combinations observed in both tables
- `outer`: Use all possible keys combinations
- `left:` Use all keys found in the first data frame
- `right:`Use all keys found in the second data frame

`Use this image as a reference`
![types of join](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)

In [24]:
homework = pd.DataFrame({
    'Semester': ['Fall 2018', 'Fall 2018', 'Fall 2019', 'Fall 2019'],
    'Name': ['Alice', 'Bob', 'Clare', 'Alice'],
    'Hw1': [50, 90, 80, 70],
    'Hw2': [60, 70, 80, 90]
})
homework

,Semester,Name,Hw1,Hw2
0,Fall 2018,Alice,50,60
1,Fall 2018,Bob,90,70
2,Fall 2019,Clare,80,80
3,Fall 2019,Alice,70,90


In [25]:
exam = pd.DataFrame({
    'When': ['Fall 2018', 'Fall 2018', 'Fall 2019', 'Fall 2019'],
    "Name": ['Alice', 'Bob', 'Clare', 'Alice'],
    "Midterm": [60, 80, 90, 100],
    "Final": [45, 65, 75, 55]
})
exam

,When,Name,Midterm,Final
0,Fall 2018,Alice,60,45
1,Fall 2018,Bob,80,65
2,Fall 2019,Clare,90,75
3,Fall 2019,Alice,100,55


In [26]:
pd.merge(homework, exam, on='Name')

,Semester,Name,Hw1,Hw2,When,Midterm,Final
0,Fall 2018,Alice,50,60,Fall 2018,60,45
1,Fall 2018,Alice,50,60,Fall 2019,100,55
2,Fall 2019,Alice,70,90,Fall 2018,60,45
3,Fall 2019,Alice,70,90,Fall 2019,100,55
4,Fall 2018,Bob,90,70,Fall 2018,80,65
5,Fall 2019,Clare,80,80,Fall 2019,90,75


In this case, since _homework_ and _exam_ both share multiple column with the same type, we can specify the column we want to use for the merge using the `on` argument.

**Note:** `on` can take either a `str` or `[str]`

In [27]:
pd.merge(homework, exam, on='Name', suffixes=('_hw', '_ex'), how='outer')

,Semester,Name,Hw1,Hw2,When,Midterm,Final
0,Fall 2018,Alice,50,60,Fall 2018,60,45
1,Fall 2018,Alice,50,60,Fall 2019,100,55
2,Fall 2019,Alice,70,90,Fall 2018,60,45
3,Fall 2019,Alice,70,90,Fall 2019,100,55
4,Fall 2018,Bob,90,70,Fall 2018,80,65
5,Fall 2019,Clare,80,80,Fall 2019,90,75


**suffixes** `[str]` : appends a suffix to the columns used for the merge

### Concatenating NumPy Arrays

This is not very important, but I'll put it here anywys. 

- `np.arrange()` creates a numpy array of numbers. 

- `.reshape()` changes the structure (shape) of the numpy array 

- `np.vstack` concatenates two numpy arrays vertically.

- `np.hstack` concatenates two numpy arrays horizontally.

In [28]:
#np.arrange(end)
arr1 = np.arange(12).reshape([3, 4])
print(arr1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [29]:
#np.arrange(start, end, step)

arr2 = np.arange(10, 90, 10).reshape([2, 4])
print(arr2)

[[10 20 30 40]
 [50 60 70 80]]


In [30]:
print(np.vstack([arr1, arr2]))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [10 20 30 40]
 [50 60 70 80]]


In [31]:
arr3 = np.arange(100, 10, -10).reshape([3, 3])
print(arr3)

[[100  90  80]
 [ 70  60  50]
 [ 40  30  20]]


In [32]:
print(np.hstack([arr1, arr3]))

[[  0   1   2   3 100  90  80]
 [  4   5   6   7  70  60  50]
 [  8   9  10  11  40  30  20]]


### Concatenating Data Frames

In [33]:
spring_records = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare', 'David'],
    'Homework': [60, 70, 80, 90],
    'Exam': [65, 75, 85, 95]
})
spring_records

,Name,Homework,Exam
0,Alice,60,65
1,Bob,70,75
2,Clare,80,85
3,David,90,95


In [34]:
fall_records = pd.DataFrame({
    'Name': ['Alice', 'Eva', 'Fred', 'Gabriel'],
    'Homework': [66, 77, 88, 99],
    'Exam': [69, 79, 89, 99]
})
fall_records

,Name,Homework,Exam
0,Alice,66,69
1,Eva,77,79
2,Fred,88,89
3,Gabriel,99,99


In [35]:
pd.concat([spring_records, fall_records])

,Name,Homework,Exam
0,Alice,60,65
1,Bob,70,75
2,Clare,80,85
3,David,90,95
0,Alice,66,69
1,Eva,77,79
2,Fred,88,89
3,Gabriel,99,99


By default, `pd.concat` stacks pandas dataframes vertically, but by passing the `axis` argument, you can specify how pandas concatenates the dataframes. `axis=0` concatenates vertically and `axis=1` concatenates horizontally. 

# Group By

In [36]:
airports = pd.read_csv("./Data/airports.csv", sep=",")
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [37]:
# Create a data frame that shows the number of large airports for each country.

# 1. Extract large airports
filter1 = (airports['type'] == 'large_airport')
large_airports = airports[filter1]
large_airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
11955,67,AYPY,large_airport,Port Moresby Jacksons International Airport,-9.443380,147.220001,146.0,OC,PG,PG-NCD,Port Moresby,yes,AYPY,POM,NaN,NaN,https://en.wikipedia.org/wiki/Jacksons_Interna...,NaN
12420,123,BIKF,large_airport,Keflavik International Airport,63.985001,-22.605600,171.0,EU,IS,IS-2,Reykjavík,yes,BIKF,KEF,NaN,https://www.isavia.is/en/keflavik-airport,https://en.wikipedia.org/wiki/Keflav%C3%ADk_In...,"Keflavik Naval Air Station,REK"
12467,4614,BKPR,large_airport,Priština International Airport,42.572800,21.035801,1789.0,EU,XK,XK-01,Prishtina,yes,BKPR,PRN,NaN,http://www.airportpristina.com/,https://en.wikipedia.org/wiki/Pristina_Interna...,"LYPR, Pristina, Slatina Air Base"
16122,329666,CN-0083,large_airport,Guodu air base,36.001741,117.632010,NaN,AS,CN,CN-U-A,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
17271,1717,CYEG,large_airport,Edmonton International Airport,53.309700,-113.580002,2373.0,NaN,CA,CA-AB,Edmonton,yes,CYEG,YEG,CYEG,http://www.edmontonairports.com/,https://en.wikipedia.org/wiki/Edmonton_Interna...,YEA


In [49]:
# 2. Split the data frame according to the country they belong to

groups = large_airports.groupby('iso_country')

The pandas `groupby` method takes as an argument one or more column names, values or functions for which pandas will separate into different groups. For instance, in the case above, Pandas will separate the data according to the iso country code 

In [48]:
num_airports = groups.size().to_frame(name="Large airports by iso country")
num_airports

,Large airports by iso country
iso_country,
AE,4
AL,1
AM,1
AO,1
AR,1
...,...
VN,3
XK,1
ZA,4


`to_frame` converts a pandas series into a dataframe

- **name** `str`: column name for the series in the group.

### Data Aggregations

Aggregation refer to any data transformation that produces numeric values from arrays. Examples of data aggregation methods include mean(), size(), count(), first(), min(), max(), and sum(). Moreover, user-defined functions can also be applied to create desired summary.

In [52]:
data = pd.read_csv("./Data/phone_data.csv", sep=",", index_col='index')
print(data.shape)
data.head(3)

(830, 6)


,date,duration,item,month,network,network_type
index,,,,,,
0,15/10/14 06:58,34.429,data,2014-11,data,data
1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile


In [53]:
# Define function get_range() that returns(max - min)
def get_range(array):
    return array.max() - array.min()

In [54]:
# Apply agg() to find the range of each type of cell phone use.
data.groupby(['item'])['duration'].agg(get_range)

item
call    10527.0
data        0.0
sms         0.0
Name: duration, dtype: float64

Aggregate using one or more operations over the specified axis. Takes in a function, lambda or list of functions, for example:

In [55]:
# Apply multiple aggregation functions
data.groupby(['item'])['duration'].agg([get_range, np.max, np.min])

,get_range,amax,amin
item,,,
call,10527.0,10528.000,1.000
data,0.0,34.429,34.429
sms,0.0,1.000,1.000


## GroupBy reminder:

groupby groups have the following methods:
mean(), size(), count(), first(), min(), max(), and sum().

And can be used in the following way:
groups.mean()
groups.size()
...

# Time Series Data


In [64]:
print(datetime.now())
print(datetime.now().year)
print(datetime.now().day)
print(datetime.now().month)

2020-12-15 23:52:16.893960
2020
15
12


In [65]:
delta = timedelta(10)

datetime.now() + delta

datetime.datetime(2020, 12, 25, 23, 52, 28, 122714)

In [70]:
# datetime to string
date = datetime(2011, 12, 3, 23, 30, 45)
str(date)

'2011-12-03 23:30:45'

`datetime` creates a datetime object 

**ARGUMENT ORDER IS IMPORTANT** <br>
datetime(year, month, day, hour, minute, second)

In [68]:
# Convert to format "YYYY-MM-DD"
date.strftime("%Y/%m/%d %H:%M, %A")

'2011/01/03 23:30, Monday'

`strftime` formats a datetime object according to the patter passed 

**SEPARATE EACH TOKEN WITH** `%`
Datetime formats:
- %Y: Four-digit year
- %y: Two-digit year
- %m: Two-digit month
- %d: Two-digit day
- %H: Hour 0 - 23
- %I: Hour 1 - 12
- %M: Two-digit minute
- %S: Second
- %A: Weekday

[More on this](https://docs.python.org/2/library/datetime.html)

In [72]:
parse("2011-01-03")

datetime.datetime(2011, 1, 3, 0, 0)

`parse` converts a string into a datetime object

In [73]:
# Many countries use format "DD/MM/YYYY". We need to set dayfirst=True
# so that the date is correctly recognized.
parse("06/12/2011", dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

### Time Series Basics 

In [74]:

# Create a list of datetime objects
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 2, 7), datetime(2011, 2, 8),
         datetime(2011, 3, 10), datetime(2011, 3, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02    1.425658
2011-01-05   -0.693034
2011-02-07   -1.425432
2011-02-08    1.717807
2011-03-10    0.482319
2011-03-12   -0.374166
dtype: float64

The cell above creates a pandas `series` object with random numbers and dates as an index. 

In this times series, we can search items in the series by date or by index, as we will see below.

In [75]:
# Select 01/05
ts['2011-01-05']

-0.693034422960849

In [76]:
ts[1]


-0.693034422960849

In [77]:
# Select a range of dates
ts['2011-02']

2011-02-07   -1.425432
2011-02-08    1.717807
dtype: float64

the cell above selects all of the object which happened on 2011-02, i.e, February 2011, for example:

- 2011-02-07    0.271973
- 2011-02-08    1.349570

In [79]:
# You can select a range of dates as well 
ts['2011-02-01':'2011-02-8'] # the end datetime is also included

2011-02-07   -1.425432
2011-02-08    1.717807
dtype: float64

### Date Ranges

In [80]:

# manually populate a list of dates
dates = [datetime(2011, 1, 2), datetime(2011, 3, 10), datetime(2011, 4, 1)]
# ts[dates] # Pandas no longer supports missing indices
ts[ts.index.isin(dates)]

2011-01-02    1.425658
2011-03-10    0.482319
dtype: float64

Remember that the pandas `.isin` method searches for any coinciding elements between a dataframe and the argument

In [81]:
# Create a range of dates
daterange = pd.date_range('2011-01-01', periods=8)

`data_range` return a DatetimeIndex object, which is like a pandas series. This object contains a range of dates starting at the specified date (`2011-01-01` in this case), with a length of `n` periods (`8` in this case). 

By default, the periods will equal 1 day, but this can be changed using the `freq` argument, for example:

In [82]:
# 2 day periods

daterange = pd.date_range('2011-01-01', periods=5, freq='2D')
print(daterange)


DatetimeIndex(['2011-01-01', '2011-01-03', '2011-01-05', '2011-01-07',
               '2011-01-09'],
              dtype='datetime64[ns]', freq='2D')


In [83]:
# 10 hour periods

daterange = pd.date_range("2011-01-01", periods=5, freq="10H")
print(daterange)

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 10:00:00',
               '2011-01-01 20:00:00', '2011-01-02 06:00:00',
               '2011-01-02 16:00:00'],
              dtype='datetime64[ns]', freq='10H')


In [84]:
# Sample business days only

daterange = pd.date_range("2011-01-01", periods=10, freq="B")
print(daterange)

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14'],
              dtype='datetime64[ns]', freq='B')
